# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120922e+02     1.431902e+00
 * time: 0.8017950057983398
     1     1.032434e+01     9.097283e-01
 * time: 2.206709861755371
     2    -1.196531e+01     9.237721e-01
 * time: 2.9528048038482666
     3    -3.403873e+01     7.247340e-01
 * time: 4.014460802078247
     4    -4.754778e+01     5.137786e-01
 * time: 5.0277488231658936
     5    -5.678398e+01     2.161218e-01
 * time: 6.052175998687744
     6    -5.970145e+01     1.048394e-01
 * time: 6.769385814666748
     7    -6.082019e+01     5.293212e-02
 * time: 7.496100902557373
     8    -6.123552e+01     5.948077e-02
 * time: 8.18645191192627
     9    -6.152069e+01     3.570161e-02
 * time: 8.884228944778442
    10    -6.175345e+01     2.631654e-02
 * time: 9.597704887390137
    11    -6.195547e+01     2.309324e-02
 * time: 10.296661853790283
    12    -6.203558e+01     1.980681e-02
 * time: 11.000824928283691
    13    -6.212001e+01     1.302563e-02
 * time: 11.689939975738525
  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671075
    AtomicLocal         -18.8557672
    AtomicNonlocal      14.8522631
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485376 
    Xc                  -19.3336822

    total               -62.261666459325
